In [1]:
import model as m
import wrangle as w
from importlib import reload
from sklearn.linear_model import LinearRegression
import warnings
warnings.simplefilter('ignore')

In [2]:
train,validate,test = w.tvt_split(w.wrangle_zillow())

In [3]:
reload(m)
m.select_baseline(train.log_error)

|                 |     SSE |       MSE |     RMSE |
|:----------------|--------:|----------:|---------:|
| Mean Baseline   | 1285.55 | 0.0296687 | 0.172246 |
| Median Baseline | 1289.55 | 0.0297612 | 0.172514 |
### Because mean outperformed median on all metrics,         we will use mean as our baseline

In [23]:
reload(m)
train_process_frame = train[['fireplace_count','latitude','longitude','tax_value','calc_sqft','bed_count','bath_count','log_error']]
features = ['fireplace_count','bed_count','bath_count']
cluster_cols = ['tax_value','calc_sqft','latitude','longitude']
cluster_name = 'cluster_fuck'
target = 'log_error'
# log_error_min = train_process_frame.log_error.min()
# train_process_frame.log_error = train_process_frame.log_error - log_error_min
clustered_scaled,scaler,clusterer = m.scale_and_cluster(train_process_frame,features=features,target=target,cluster_cols=cluster_cols,cluster_name=cluster_name,k=7)
regressors = m.generate_regressor(clustered_scaled,features=features,target=target,cluster_col=cluster_name,regressor=LinearRegression(normalize=True))
ypred = m.apply_to_clusters(features=clustered_scaled,target=target,cluster_col=cluster_name,regressor=regressors)

IndexError: positional indexers are out-of-bounds

In [ ]:
predictions_df = pd.DataFrame()
predictions_df['y_true'] = train.log_error
predictions_df['y_pred'] = 1.0
cluster_group = clustered_scaled.groupby('cluster_fuck')
for i,group in cluster_group:
    predictions_df.iloc[group.index,1] = regressors[i].predict(group)
predictions_df.y_true = predictions_df.y_true + log_error_min
predictions_df.y_pred = predictions_df.y_pred + log_error_min


In [ ]:
import evaluate as ev
ev.regression_errors(predictions_df.y_true,predictions_df.y_pred,'Regression Model')

,SSE,ESS,TSS,MSE,RMSE
Regression Model,1303.639804,22.013285,1325.653089,0.030086,0.173454
